# Đồ Án Cuối Kì Môn Khoa Học Dữ Liệu

**Giới thiệu thành viên**

- *Họ tên:* **Trần Ngọc Tịnh**       
  *MSSV:* **18120597**
  
  
  
  
- *Họ tên:* **Nguyễn Ngọc Năng Toàn**       
  *MSSV:* **18120600**

## Import 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

# You can also import other things ...
# YOUR CODE HERE (OPTION)

## Đọc dữ liệu từ file csv

In [2]:
data_df = pd.read_csv('data.csv', 
                      index_col=0) # Cho cột index là cột 0 (PassengerId)
data_df.head()

,confirmed,recovered,deaths,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city,lat,long,updated
Country,,,,,,,,,,,,,,,
Afghanistan,51526,41727,2191,35530081.0,652090.0,45.9,NaN,Asia,AF,Southern and Central Asia,4.0,Kabul,33.93911,67.709953,2021/01/04 05:22:02+00
Albania,59438,34648,1193,2930187.0,28748.0,71.6,NaN,Europe,AL,Southern Europe,8.0,Tirana,41.15330,20.168300,2021/01/04 05:22:02+00
Algeria,100408,67808,2772,41318142.0,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger,28.03390,1.659600,2021/01/04 05:22:02+00
Andorra,8192,7517,84,76965.0,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella,42.50630,1.521800,2021/01/04 05:22:02+00
Angola,17642,11223,408,29784193.0,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda,-11.20270,17.873900,2021/01/04 05:22:02+00


In [3]:
data_df.index.duplicated().sum()

0

In [4]:
data_df['deaths'].dtype

dtype('int64')

In [5]:
data_df['deaths'].isna().sum()

0

Ta thấy dữ liệu cột continent bị thiếu giá trị.

In [6]:
data_df['deaths'].value_counts(normalize=True) * 100

0        7.558140
3        1.162791
5        1.162791
1        1.162791
65       1.162791
           ...   
4957     0.581395
1374     0.581395
352      0.581395
10350    0.581395
7678     0.581395
Name: deaths, Length: 153, dtype: float64

In [7]:
y_sr = data_df["deaths"] # sr là viết tắt của series
X_df = data_df.drop("deaths", axis=1)

Tiền xử lý tập y_sr

In [8]:
values=y_sr.values
values

array([  2191,   1193,   2772,     84,    408,      5,  43482,   2850,
          909,   6324,   2733,    175,    352,   7626,      7,   1451,
        19701,    250,     44,      0,   9201,   4131,     42, 196018,
            3,   7678,     86,      2,      0,    448,  15880,     63,
          104,  16767,   4784,  43965,     13,   2185,   4072,    147,
          131,   1374,     61,      0,   2418,  14059,   7805,   1358,
           86,      1,    244,   1948,    561,  65164,     64,    124,
         2603,  34791,    335,   4957,      0,   4833,     81,     45,
          164,    236,   3180,   9884,     29, 149649,  22734,  55540,
        12834,   2259,   3416,  75332,    304,   3429,   3903,   2771,
         1685,    937,   1360,      0,    680,   1499,     65,     83,
           39,   1643,    495,   2530,    262,    195,    494,     48,
          278,    220,      0,    358,     10, 127213,   3037,      3,
            1,    689,   7485,    169,    215,   1878,  11707,     25,
      

In [9]:
#Xu ly y_Sr
values[values<100]=0
values[(values>=100)&(values<1000)]=1
values[values>=1000]=2
values


array([2, 2, 2, 0, 1, 0, 2, 2, 1, 2, 2, 1, 1, 2, 0, 2, 2, 1, 0, 0, 2, 2,
       0, 2, 0, 2, 0, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 0,
       2, 2, 2, 2, 0, 0, 1, 2, 1, 2, 0, 1, 2, 2, 1, 2, 0, 2, 0, 0, 1, 1,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 0, 1, 2, 0, 0,
       0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 0, 1, 2, 1, 1, 2,
       2, 0, 1, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 0, 0, 0,
       0, 0, 2, 1, 2, 0, 0, 0, 2, 2, 0, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 0,
       0, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 1, 0, 2, 0, 1, 1, 1], dtype=int64)

In [10]:
y_sr

Country
Afghanistan    2
Albania        2
Algeria        2
Andorra        0
Angola         1
              ..
Venezuela      2
Vietnam        0
Yemen          1
Zambia         1
Zimbabwe       1
Name: deaths, Length: 172, dtype: int64

Tiến hành chi tập theo tỷ lệ 7:3

In [11]:
train_X, test_X, train_y, test_y = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

In [12]:
train_X.location.value_counts()

Middle East                  11
Caribbean                    10
South America                10
Southern and Central Asia    10
Eastern Africa               10
Western Africa                9
Southern Europe               8
Western Europe                7
Eastern Europe                6
Central Africa                5
Nordic Countries              5
Southeast Asia                5
Northern Africa               5
Southern Africa               4
Central America               4
Melanesia                     3
Baltic Countries              2
Eastern Asia                  2
Polynesia                     1
British Isles                 1
Name: location, dtype: int64

Tạo class ColAdderDroppep để xóa các cột ko cần thiết

In [13]:
class ColDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        df = X_df.copy()
        unused_cols = ['abbreviation', 'capital_city', 'updated','sq_km_area','location']
        df=df.drop(unused_cols,axis=1)
        df=df.astype({'confirmed':'float64','recovered':'float64'})
        df['elevation_in_meters'] = pd.to_numeric(df['elevation_in_meters'],errors='coerce')
        return df

In [14]:
col_dropper = ColDropper()
col_dropper.fit(train_X)
col_dropper.transform(train_X).head()

,confirmed,recovered,population,life_expectancy,elevation_in_meters,continent,iso,lat,long
Country,,,,,,,,,
Cyprus,23974.0,2057.0,1179551.0,76.7,91.0,Asia,196.0,35.126400,33.429900
Tanzania,509.0,183.0,57310019.0,52.3,NaN,Africa,834.0,-6.369028,34.888822
Botswana,14805.0,12136.0,2291661.0,39.3,NaN,Africa,72.0,-22.328500,24.684900
Morocco,443146.0,413393.0,35739580.0,69.1,909.0,Africa,504.0,31.791700,-7.092600
Togo,3702.0,3474.0,7797694.0,54.7,236.0,Africa,768.0,8.619500,0.824800


Tạo pipeline tiền xử lý

In [15]:
nume_cols=['confirmed','recovered','life_expectancy','elevation_in_meters','iso','lat','long']
cate_cols=['continent']

mean = SimpleImputer(strategy='mean')
mode = SimpleImputer(strategy='most_frequent') 
mode_encode = make_pipeline(mode,OneHotEncoder(handle_unknown='ignore'))

colTransform = ColumnTransformer([('mean_for_nume_cols',mean, nume_cols),
                                ('mode_encode_for_cate_cols',mode_encode,cate_cols)])
colNormalize = StandardScaler()
neural_net_model = MLPClassifier(hidden_layer_sizes=(20,), activation='tanh',
                                 solver='lbfgs', max_iter=2500, random_state=0)
full_pipeline = make_pipeline(col_dropper,colTransform,colNormalize,neural_net_model)

print(full_pipeline)

Pipeline(steps=[('coldropper', ColDropper()),
                ('columntransformer',
                 ColumnTransformer(transformers=[('mean_for_nume_cols',
                                                  SimpleImputer(),
                                                  ['confirmed', 'recovered',
                                                   'life_expectancy',
                                                   'elevation_in_meters', 'iso',
                                                   'lat', 'long']),
                                                 ('mode_encode_for_cate_cols',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
               

In [16]:
full_pipeline.fit(train_X, train_y)
Y_pred = full_pipeline.predict(test_X)

Đưa vào datafame dữ liệu dự đoán và dữ liệu gốc

In [17]:
result={'predict':Y_pred,'actual':test_y}
df=pd.DataFrame(result)
print(df)

                     predict  actual
Country                             
Gabon                      0       0
Spain                      2       2
Guinea-Bissau              1       0
Gambia                     1       1
Qatar                      1       1
Mauritius                  0       0
Armenia                    2       2
Timor-Leste                0       0
Mongolia                   0       0
Lithuania                  2       2
Sierra Leone               0       0
Somalia                    0       1
Antigua and Barbuda        0       0
Malta                      0       1
United Kingdom             2       2
Malawi                     1       1
Chile                      2       2
Benin                      0       0
Guatemala                  2       2
Ethiopia                   1       2
Honduras                   0       2
Costa Rica                 2       2
Angola                     0       1
Panama                     2       2
Grenada                    0       0
N

Tỷ lệ phần trăm dự đoán chính xác

In [18]:
print(round((1-df[df.predict!=df.actual].shape[0]/df.shape[0])*100,3),"%")

75.0 %
